In [ ]:
from common_preamble import *

In [ ]:
mm = trimesh_io.MeshMeta(disk_cache_path=mesh_dir,
                         cache_size=0, cv_path=mesh_cv_path, voxel_scaling=voxel_scaling)

soma_meshes = [] 
ais_meshes = []
for oid in complete_ais_ids:
    soma_file = ais_mesh_dir + '/{}_soma.h5'.format(oid)
    actors = []
    if os.path.exists(soma_file):
        soma_mesh = mm.mesh(filename=soma_file)
        soma_meshes.append(soma_mesh)
    else:
        soma_meshes.append(None)
        print('{} soma not found!'.format(oid))
    ais_file = ais_mesh_dir + '/{}_ais.h5'.format(oid)
    if os.path.exists(soma_file): 
        ais_mesh = mm.mesh(filename=ais_file)
        ais_meshes.append(ais_mesh)
    else:
        ais_meshes.append(None)
        print('{} AIS not found!'.format(oid))
    

In [ ]:
for ais_mesh, soma_mesh, oid in tqdm.tqdm(zip(ais_meshes, soma_meshes, complete_ais_ids), total=len(complete_ais_ids)):
    if ais_mesh is None or soma_mesh is None:
        continue
    soma_actor = trimesh_vtk.mesh_actor(soma_mesh, color=non_color, opacity=1)
    ais_actor = trimesh_vtk.mesh_actor(ais_mesh, color=ais_color, opacity=1)
    ctr_pt = (soma_mesh+ais_mesh).bounding_box.center_mass
    actors = [soma_actor, ais_actor]
              
    camera = trimesh_vtk.oriented_camera(ctr_pt, backoff=200)
    trimesh_vtk.render_actors(actors, camera=camera, back_color=(1,1,1), scale=4,
                                do_save=True, filename=plot_dir + '/ais_views/soma_ais_{}.png'.format(oid),
                                VIDEO_WIDTH=800, VIDEO_HEIGHT=1200)

In [ ]:
bad_mesh = ais_meshes[np.flatnonzero(complete_ais_ids==648518346349519354)[0]]

In [ ]:
from meshparty import skeletonize

In [ ]:
sk = skeletonize.skeletonize_mesh(bad_mesh, collapse_soma=False, invalidation_d=5000)

In [ ]:
ma = trimesh_vtk.mesh_actor(bad_mesh)
sa = trimesh_vtk.skeleton_actor(sk, color=(0,0,0), line_width=2)
trimesh_vtk.render_actors([ma, sa])

In [ ]:
mm2 = trimesh_io.MeshMeta(disk_cache_path='.',
                         cache_size=0, cv_path=mesh_cv_path, voxel_scaling=voxel_scaling)

---
#### Save masks

Note: May want to gzip the resulting h5 at the end. Using the dataframe this way does not engage the compression in pandas very naturally, so the file ends up being much bigger than it needs to be.

In [ ]:
soma_masks = {}
ais_masks = {}
rel_oids = []
for ais_mesh, soma_mesh, oid in zip(ais_meshes, soma_meshes, complete_ais_ids):
    if ais_mesh is None or soma_mesh is None:
        continue
    rel_oids.append(oid)
    soma_masks[oid] = soma_mesh.node_mask
    ais_masks[oid] = ais_mesh.node_mask

In [ ]:
import h5py

mask_dict = {}
for oid in soma_masks.keys():
    mask_dict[oid] = {}
    mask_dict[oid]['soma'] = soma_masks[oid]
    mask_dict[oid]['ais'] = ais_masks[oid]
    
mask_filename = base_dir + '/data/in/ais_and_soma_masks_v{}.h5'.format(data_version)
with h5py.File(name=mask_filename, mode='w') as f:
    for oid in mask_dict:
        grp = f.create_group(str(oid))
        grp.create_dataset('soma', data=soma_masks[oid], compression='gzip')
        grp.create_dataset('ais', data=ais_masks[oid], compression='gzip')

---
Example loading function:

In [ ]:
def import_masks_file(fname):
    soma_masks = {}
    ais_masks = {}
    with h5py.File(fname, mode='r') as f:
        oids = f.keys()
        for oid in oids:
            soma_masks[np.int(oid)] = f[oid]['soma'][()]
            ais_masks[np.int(oid)] = f[oid]['ais'][()]
    return soma_masks, ais_masks

---
### Skeletonize AISes

In [ ]:
len(complete_ais_ids)

In [ ]:
soma_df = dl.query_cell_types(soma_table)
ids_e = soma_df[soma_df['cell_type']=='e']['pt_root_id'].values
ids_i = soma_df[soma_df['cell_type']=='i']['pt_root_id'].values

In [ ]:
ee_df = dl.query_synapses(synapse_table, pre_ids=ids_e, post_ids=ids_e)
ei_df = dl.query_synapses(synapse_table, pre_ids=ids_e, post_ids=ids_i)
ie_df = dl.query_synapses(synapse_table, pre_ids=ids_i, post_ids=ids_e)
ii_df = dl.query_synapses(synapse_table, pre_ids=ids_i, post_ids=ids_i)

In [ ]:
len(ids_e), len(ids_i)

In [ ]:
len(ee_df), len(ei_df), len(ie_df), len(ii_df)

In [ ]:
len(ee_df)